In [1]:
# 02_baseline_ml.ipynb

# ===============================
# 1. Import des librairies
# ===============================
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_recall_curve, auc

# ===============================
# 2. Chargement des données
# ===============================
X_train = pd.read_csv('../data/processed/X_train.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
y_train = pd.read_csv('../data/processed/y_train.csv')
y_test = pd.read_csv('../data/processed/y_test.csv')

# Convertir y_train et y_test en Series
y_train = y_train.iloc[:,0]
y_test = y_test.iloc[:,0]

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# ===============================
# 3. Initialisation et entraînement du modèle
# ===============================
baseline_model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
baseline_model.fit(X_train, y_train)

# ===============================
# 4. Prédiction
# ===============================
y_pred = baseline_model.predict(X_test)
y_pred_proba = baseline_model.predict_proba(X_test)[:,1]

# ===============================
# 5. Évaluation
# ===============================
# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

# AUPRC (Area Under Precision-Recall Curve)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
auprc = auc(recall, precision)
print(f"AUPRC: {auprc:.4f}")




X_train shape: (227845, 30), y_train shape: (227845,)
X_test shape: (56962, 30), y_test shape: (56962,)
Classification Report:
              precision    recall  f1-score   support

           0     0.9999    0.9756    0.9876     56864
           1     0.0609    0.9184    0.1141        98

    accuracy                         0.9755     56962
   macro avg     0.5304    0.9470    0.5509     56962
weighted avg     0.9982    0.9755    0.9861     56962

AUPRC: 0.7639


# Analyse des résultats du modèle Logistic Regression

## 1. Taille des jeux de données
- **X_train** : `(227845, 30)`  
- **X_test** : `(56962, 30)`  

> C’est cohérent avec le dataset de Kaggle (28 composantes PCA + `Time` et `Amount` = 30 features).  

---

## 2. Classification report
- **Classe 0 (non fraude)** :  
  - Precision ≈ 0.9999 → presque toutes les transactions normales sont correctement identifiées.  
  - Recall ≈ 0.9756 → environ 2,4% des transactions normales sont mal classées.  

- **Classe 1 (fraude)** :  
  - Precision ≈ 0.0609 → beaucoup de faux positifs pour la classe fraude.  
  - Recall ≈ 0.9184 → le modèle détecte ~92% des fraudes, ce qui est excellent.  

- **F1-score** pour la fraude ≈ 0.1141 → très faible à cause de la precision faible.  

> Cela montre un déséquilibre typique : le modèle détecte bien la majorité des fraudes (recall élevé) mais produit beaucoup de faux positifs (precision faible).  

---

## 3. Accuracy
- Accuracy = 0.9755 → très élevée, mais **trompeuse** sur un dataset déséquilibré.  
- Ici, 99,8% des transactions sont normales, donc même un modèle naïf qui prédit toujours “non fraude” aurait ~99% accuracy.  

---

## 4. AUPRC
- AUPRC ≈ 0.7639 → plutôt bon pour ce dataset déséquilibré.  
- L’AUPRC est plus significatif que l’accuracy ici, car il mesure la capacité du modèle à détecter les fraudes tout en limitant les faux positifs.  

---

## Conclusion
**Points forts :**  
- Recall très élevé pour la fraude → le modèle détecte la majorité des fraudes.  
- AUPRC correct → le modèle est utile pour la détection.  

**Points faibles :**  
- Precision très faible → beaucoup de faux positifs, ce qui peut être gênant en production (transactions normales bloquées).  

---

## Améliorations possibles
1. **Équilibrer le dataset** : SMOTE, undersampling ou oversampling.  
2. **Essayer d’autres modèles** : Random Forest, Gradient Boosting, XGBoost.  
3. **Ajuster le threshold** pour réduire les faux positifs.  
4. **Modèles coût-sensibles** (le coût d’un faux négatif est élevé).  
5. **Ensemble methods** : combiner plusieurs modèles pour améliorer precision et recall simultanément.
